# Setup

In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline
import src.advection_schemes as advection_schemes
import src.advection_tests as advection_tests
import src.config as config 
import src.animation as animation 
import matplotlib.pyplot as plt 

import numpy as np

c = config.generateConfig(dt=1e-4, t0=0)

def resetConfig(c):
    c["dt"] = 1e-3
    c["domainSize"] = 5
    c["xlim"] =  [0, 5]
    c["densityYlim"] = [0, 1.6]
    c["resolution"] = 256
    c["timeOrder"] = 1
    c["stencilOrder"] = 2
    c["dimension"] = 1
    c["debug"] = False
    c["slowDown"] = 1
    c["tEnd"] = 1
    c["gravity"] = 1
    c["outputTimestep"] = False
    c["alpha"] = 0.1
    c["useAdaptiveTimestep"] = True
    c["usePeriodicBC"] = False
    c["dpi"] = 200
    c["size"] = (3.54 * 2, 3.54)#(3.54 * 1.5, 3.54)
    c["savePlots"] = True
    
resetConfig(c)


def run(title, scheme, c, test):
    solver = scheme(c, test)
    i = 0
    while (solver.t < solver.config["tEnd"]):
        dt = solver.getTimeStep()
        if dt + solver.t > solver.config["tEnd"]:
            dt = solver.config["tEnd"] - solver.t
        solver.step(dt)
        i += 1

    print(f"Finished in {i} steps.")
    
    animation.drawPrettyFrame(solver, advection=True)
    plt.show()

# Chapter 4

## The linear advection equation

In [7]:
algorithms = {
    "centered difference":advection_schemes.CenteredDifferenceScheme,
    "upwind": advection_schemes.UpwindScheme,
    "second-order upwind", advection_schemes.SecondOrderUpwindScheme
}

tests = {
    "gaussian": advection_tests.gaussian1D,
    "tophat": advection_tests.tophat1D
}

def f(t, resolution, scheme, test, take_snapshot):
    c["resolution"] = resolution
    c["tEnd"] = t
    c["savePlots"] = take_snapshot
    run(scheme, algorithms[scheme], c, tests[test])

interactive_plot = interactive(f, 
t=widgets.FloatSlider(min=0, max=3, step=0.1, value=1.5), 
resolution=widgets.IntSlider(min=16, max=256, step=4, value=128), 
scheme = ["centered difference", "upwind", "second-order upwind"],
test = ["gaussian", "tophat"],
take_snapshot = False
)
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=1.5, description='t', max=3.0), IntSlider(value=128, description='reso…